In [31]:
# Install a pip package in the current Jupyter kernel
import sys
sys.path.append('../wdtools')

### General
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from tqdm import tqdm
import pandas as pd

### Query
from astropy.io import fits
import wdtools

In [32]:
wide_binaries = pd.read_csv('data/wide_binaries.csv')

In [33]:
import time

wd_flux = []
wd_wavelength = []
wd_ivar = []

ms_flux = []
ms_wavelength = []
ms_ivar = []

for i in tqdm( range(len(wide_binaries))):
    try:
        wd_spec = fits.open(wide_binaries['url'][i], allow_insecure=True)
        ms_spec = fits.open(wide_binaries['mainseq_url'][i], allow_insecure=True)
        
        wd_flux.append(wd_spec[1].data['flux'])
        wd_wavelength.append(10**wd_spec[1].data['loglam'])
        wd_ivar.append(wd_spec[1].data['ivar'])
        
        ms_flux.append(ms_spec[1].data['flux'])
        ms_wavelength.append(10**ms_spec[1].data['loglam'])
        ms_ivar.append(ms_spec[1].data['ivar'])
        
        time.sleep(2)
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [01:17<00:00,  2.35s/it]


In [34]:
sp = wdtools.SpecTools()
h_alpha = 6562.79

wd_rv = []
wd_e_rv = []

ms_rv = []
ms_e_rv = []

for i in tqdm(range(len(wd_flux))):
    wrv, e_rv = sp.get_line_rv(wd_wavelength[i], wd_flux[i], wd_ivar[i], centroid = h_alpha,
                          distance = 100, edge = 15,
                          nmodel = 2, plot = False)
    mrv, e_rv = sp.get_line_rv(ms_wavelength[i], ms_flux[i], ms_ivar[i], centroid = h_alpha,
                          distance = 100, edge = 15,
                          nmodel = 2, plot = False)

    wd_rv.append(wrv)
    wd_e_rv.append(e_rv)
    
    ms_rv.append(mrv)
    ms_e_rv.append(ms_e_rv)
    

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [08:53<00:00, 19.04s/it]


In [35]:
print(wd_rv)
print(ms_rv)
print(np.array([wd_rv]) - np.array([ms_rv]))

[108.08080808080805, 120.20202020202018, 148.48484848484847, 714.1414141414141, 166.66666666666657, 182.82828282828285, 158.58585858585857, 120.20202020202018, 110.10101010101008, 144.44444444444446, 31.313131313131265, 51.51515151515136, 49.49494949494934, 134.3434343434342, 120.20202020202018, -1100.0, -31.31313131313138, 89.89898989898975, 130.30303030303017, 150.5050505050505, 102.020202020202, 166.66666666666657, 59.59595959595944, -59.595959595959656, 190.9090909090909, 108.08080808080805, 944.4444444444443, 162.62626262626253]
[146.46464646464648, 168.68686868686856, 128.28282828282815, 156.56565656565655, 152.5252525252525, -25.252525252525317, -358.5858585858586, -449.49494949494954, 128.28282828282815, 196.96969696969697, 67.67676767676764, 53.53535353535338, 43.43434343434329, -605.0505050505051, 120.20202020202018, 108.08080808080805, 19.19191919191914, 106.06060606060605, -1100.0, -857.5757575757576, -1100.0, -1100.0, -207.07070707070704, 122.2222222222221, 247.47474747474

In [36]:
print(np.array([wd_rv]) - np.array([ms_rv]))

[[  -38.38383838   -48.48484848    20.2020202    557.57575758
     14.14141414   208.08080808   517.17171717   569.6969697
    -18.18181818   -52.52525253   -36.36363636    -2.02020202
      6.06060606   739.39393939     0.         -1208.08080808
    -50.50505051   -16.16161616  1230.3030303   1008.08080808
   1202.02020202  1266.66666667   266.66666667  -181.81818182
    -56.56565657   109.09090909   844.44444444  -781.81818182]]
